<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [70]:
import pandas as pd
import sqlite3
from contextlib import closing

In [123]:
data = pd.read_csv('blast_result_table_data.csv', sep='\t')
meta = pd.read_csv('blast_result_table_meta.csv', sep='\t')

In [124]:
data.head()

,ID,Alignment_length,Query_start,Query_end,Subject_start,Subject_end,Evalue
0,1,17,1,17,185144,185160,1.5
1,2,16,2,17,52123,52138,6.0
2,3,16,2,17,4393,4408,6.0
3,4,16,2,17,20752,20767,6.0
4,5,16,5,20,77630,77645,6.0


In [125]:
meta.head()

,ID,Query,Subject
0,1,Left_primer,CAJJJL010061470.1
1,2,Left_primer,CAJJJL010081078.1
2,3,Left_primer,CAJJJL010057850.1
3,4,Left_primer,CAJJJL010057850.1
4,5,Left_primer,CAJJJL010035108.1


In [181]:
with closing(sqlite3.connect('blast.db')) as conn:
    data.to_sql('data', conn, index=False)
    meta.to_sql('meta', conn, index=False)

In [182]:
with closing(sqlite3.connect('blast.db')) as conn:
    result = pd.read_sql('SELECT * FROM data WHERE Alignment_length=="20"', conn)
result

,ID,Alignment_length,Query_start,Query_end,Subject_start,Subject_end,Evalue
0,9,20,1,20,113544,113563,0.025
1,92,20,1,20,206747297,206747316,6.000


In [183]:
with closing(sqlite3.connect('blast.db')) as conn:
    result = pd.read_sql('SELECT * FROM meta WHERE Subject=="CAJJJL010061470.1"', conn)
result

,ID,Query,Subject
0,1,Left_primer,CAJJJL010061470.1


In [184]:
with closing(sqlite3.connect('blast.db')) as conn:
    conn.cursor().execute('UPDATE data SET Query_end="9999" WHERE ID=="1"')
    conn.commit()
    result = pd.read_sql('SELECT * FROM data WHERE ID=="1"', conn)
result

,ID,Alignment_length,Query_start,Query_end,Subject_start,Subject_end,Evalue
0,1,17,1,9999,185144,185160,1.5


In [185]:
with closing(sqlite3.connect('blast.db')) as conn:
    result = pd.read_sql('SELECT * FROM data', conn)
result

,ID,Alignment_length,Query_start,Query_end,Subject_start,Subject_end,Evalue
0,1,17,1,9999,185144,185160,1.5
1,2,16,2,17,52123,52138,6.0
2,3,16,2,17,4393,4408,6.0
3,4,16,2,17,20752,20767,6.0
4,5,16,5,20,77630,77645,6.0
...,...,...,...,...,...,...,...
130,131,16,3,18,377470,377455,6.0
131,132,16,3,18,278444,278429,6.0
132,133,16,3,18,20575,20560,6.0
133,134,16,1,16,225401520,225401505,6.0


In [186]:
with closing(sqlite3.connect('blast.db')) as conn:
    conn.cursor().execute('DELETE FROM data WHERE Query_start=="1"')
    conn.commit()
    result = pd.read_sql('SELECT * FROM data', conn)
result

,ID,Alignment_length,Query_start,Query_end,Subject_start,Subject_end,Evalue
0,2,16,2,17,52123,52138,6.0
1,3,16,2,17,4393,4408,6.0
2,4,16,2,17,20752,20767,6.0
3,5,16,5,20,77630,77645,6.0
4,6,16,2,17,520623,520608,6.0
...,...,...,...,...,...,...,...
123,130,16,3,18,38673,38658,6.0
124,131,16,3,18,377470,377455,6.0
125,132,16,3,18,278444,278429,6.0
126,133,16,3,18,20575,20560,6.0


In [187]:
conn = sqlite3.connect('blast.db')
conn.execute("PRAGMA foreign_keys = ON")
cur = conn.cursor()
cur.execute(
    '''
    CREATE TABLE table1 (
        ID INT PRIMARY KEY,
        query VARCHAR(255) NOT NULL
    )
    '''
)
cur.execute(
    '''
    CREATE TABLE table2 (
        ID INT PRIMARY KEY,
        Subject_start INT NOT NULL,
        FOREIGN KEY (ID) REFERENCES table1 (ID) ON DELETE CASCADE
    )
    '''
)
conn.commit()
conn.close()